In [1]:
spark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.appName('OlistData')\
.getOrCreate()

25/04/16 16:25:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
customer_df = spark.read.csv('/data/olist/olist_customers_dataset.csv',header=True,inferSchema=True)
geolocation_df = spark.read.csv('/data/olist/olist_geolocation_dataset.csv',header=True,inferSchema=True)
order_item_df = spark.read.csv('/data/olist/olist_order_items_dataset.csv',header=True,inferSchema=True)
payment_df = spark.read.csv('/data/olist/olist_order_payments_dataset.csv',header=True,inferSchema=True)
review_df = spark.read.csv('/data/olist/olist_order_reviews_dataset.csv',header=True,inferSchema=True)
order_df = spark.read.csv('/data/olist/olist_orders_dataset.csv',header=True,inferSchema=True)
product_df = spark.read.csv('/data/olist/olist_products_dataset.csv',header=True,inferSchema=True)
seller_df = spark.read.csv('/data/olist/olist_sellers_dataset.csv',header=True,inferSchema=True)
category_translation_df = spark.read.csv('/data/olist/product_category_name_translation.csv',header=True,inferSchema=True)

25/04/16 16:25:43 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/04/16 16:25:58 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/miniconda3/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardI

KeyboardInterrupt: 

25/04/16 16:26:13 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/04/16 16:26:28 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/04/16 16:26:43 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/04/16 16:26:58 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/04/16 16:27:13 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/04/16 16:27:28 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registere

In [32]:
# cache frequanlty  used data for better performance
order_df.cache()
customer_df.cache()
order_item_df.cache()

25/04/16 10:54:58 WARN CacheManager: Asked to cache already cached data.
25/04/16 10:54:58 WARN CacheManager: Asked to cache already cached data.
25/04/16 10:54:58 WARN CacheManager: Asked to cache already cached data.


DataFrame[order_id: string, order_item_id: int, product_id: string, seller_id: string, shipping_limit_date: timestamp, price: double, freight_value: double]

In [7]:
order_item_join_df = order_df.join(order_item_df,'order_id','inner')
order_item_product_df = order_item_join_df.join(product_df,'product_id','inner')
order_item_products_seller_df = order_item_product_df.join(seller_df , 'seller_id' , 'inner')


In [8]:
full_order_df = order_item_products_seller_df.join(customer_df,'customer_id','inner')

In [10]:
full_order_df = full_order_df.join(geolocation_df,full_order_df.customer_zip_code_prefix == geolocation_df.geolocation_zip_code_prefix,'left')

In [12]:
full_order_df = full_order_df.join(review_df,'order_id','left')


In [14]:
full_order_df = full_order_df.join(payment_df,'order_id','left')

In [15]:
full_order_df.printSchema

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [23]:
from pyspark.sql.functions import *

In [18]:
full_order_df.count()

18064261

# total revenues per seller

In [24]:
seller_revenue_df = full_order_df.groupBy('seller_id').agg(sum('price'))

In [25]:
seller_revenue_df.show(5)

+--------------------+------------------+
|           seller_id|        sum(price)|
+--------------------+------------------+
|3d5d0dc7073a299e3...|          170639.6|
|2138ccb85b11a4ec1...| 1943866.069999998|
|d650b663c3b5f6fb3...|         2253869.1|
|cd06602b43d8800bd...| 353150.9800000004|
|3c487ae8f8d7542be...|1618845.7000000055|
+--------------------+------------------+
only showing top 5 rows



In [30]:
most_sold_product = full_order_df.groupBy('product_category_name').agg(count('product_id'))

In [31]:
most_sold_product.show(2)

+---------------------+-----------------+
|product_category_name|count(product_id)|
+---------------------+-----------------+
|           perfumaria|           539288|
|       consoles_games|           180691|
+---------------------+-----------------+
only showing top 2 rows



In [39]:
# total order per customer

customer_order_count_df = full_order_df.groupBy('customer_id')\
.agg(count('order_id').alias('total_order'))\
.orderBy(desc('total_order'))

In [40]:
customer_order_count_df.show(1)

+--------------------+-----------+
|         customer_id|total_order|
+--------------------+-----------+
|351e40989da90e704...|      11427|
+--------------------+-----------+
only showing top 1 row



In [41]:
# average review score per seller
seller_reviev_df = full_order_df.groupBy('seller_id')\
.agg(avg('review_score').alias('avg_review_score'))\
.orderBy(desc('avg_review_score'))

In [42]:
seller_reviev_df.show(1)

+--------------------+----------------+
|           seller_id|avg_review_score|
+--------------------+----------------+
|228e4c1a0be164f61...|             5.0|
+--------------------+----------------+
only showing top 1 row



In [46]:
# top 10 most sold product
top_product_df = full_order_df.groupBy('product_id')\
.agg(count('order_id').alias('top_sold_product'))\
.orderBy(desc('top_sold_product'))
top_product_df.show(1)

+--------------------+----------------+
|          product_id|top_sold_product|
+--------------------+----------------+
|aca2eb7d00ea1a7b8...|           86740|
+--------------------+----------------+
only showing top 1 row



# windows function and ranking

In [47]:
from pyspark.sql.window import Window

In [49]:
# dense rank for seller based on revenue

window_spec = Window.partitionBy('seller_id').orderBy(desc('price'))

In [52]:
# rank top selling product per seller

top_selling_product_df = full_order_df.withColumn('rank',rank().over(window_spec)).filter(col('rank') <= 5)
top_selling_product_df.select('seller_id','price','rank').show()

+--------------------+-----+----+
|           seller_id|price|rank|
+--------------------+-----+----+
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
|0015a82c2db000af6...|895.0|   1|
+--------------------+-----+----+
only showing top 20 rows



<!-- advance aggregation and enrichment -->

#  advance aggregation and enrichment  

In [53]:
# total revenue and avrage order value(aov) per customer

customer_spending_df = full_order_df.groupBy('customer_id')\
.agg(
    count('order_id').alias('total_orders'),
    sum('price').alias('total_spend'),
    round(avg('price'),2).alias('AOV')
)\
.orderBy(desc('total_spend'))

customer_spending_df.show(2)

+--------------------+------------+-----------+------+
|         customer_id|total_orders|total_spend|   AOV|
+--------------------+------------+-----------+------+
|d3e82ccec3cb5f956...|        6876|  6662844.0| 969.0|
|df55c14d1476a9a34...|         743|  3565657.0|4799.0|
+--------------------+------------+-----------+------+
only showing top 2 rows

